In [2]:
# 은행(기업, 농협) 데이터 처리

import pandas as pnds
from bs4 import BeautifulSoup
import re
import pickle
import datetime

# 기업은행에서 오른쪽끝에서 두번쨰..'텍스트형식저장'한 내역을 읽고 dataframe을 만든다
with open('./downdata/20220630/거래내역조회_입출식 예금20220630.txt', 'r', encoding='euc-kr') as file:
    lines = file.readlines()
    
lines = list(map(lambda line: line.strip().split('|'), lines))      # '\n' 제거하고 '|' 단위로 분해
ibk_df = pnds.DataFrame(lines)                                      # DataFrame 생성
ibk_df.rename(columns=ibk_df.iloc[0], inplace=True)                 # 첫번쨰 행 컬럼으로 지정
ibk_df.drop(ibk_df.index[0], inplace=True)                          # 중복되는 첫번째 행 삭제

# 데이터 전처리
# ibk_df = ibk_df[[1, 3, 5, 12]]                            # 필요한 컬럼만 추출: [거래일시, 입금, 거래내용, 상대계좌예금주명]
ibk_df = ibk_df[['거래일시', '입금', '거래내용', '상대계좌예금주명']]   # 필요한 컬럼 추출
ibk_df.columns = ['date', 'receipts', 'details', 'holder']  # 컬럼명 변경
# ibk_df['date'] = ibk_df['date'].map(lambda str_data: str_data.split(' ')[0], na_action = 'ignore')  # 날짜 포맷 : '%Y-%m-%d'

# 천단위 구분자(,)를 없애야 형변환 가능했음
ibk_df['receipts'] = ibk_df['receipts'].str.replace(',', '').astype('int64', errors='ignore')  # 입금액은 'int64'형으로 변경

# holder 데이터를 카드사 이름으로 셋팅
# HD, LT, SS, SH, KEB, BC, KB (NH는 따로 셋팅)
# 이상한 문자열 ' (' 부터 ') ' 까지 문자열을 제거하고 변경하는 방식도 있음
# ibk_df['holder'] = ibk_df['holder'].str.rstrip(' （주）')
# ibk_df = ibk_df.replace({'holder': '현대카드'}, 'HD')
ibk_df = ibk_df.replace({'holder': '현대카드（주）'}, 'HD')
ibk_df = ibk_df.replace({'holder': '롯데카드（주）'}, 'LT')
ibk_df = ibk_df.replace({'holder': '삼성카드（주）'}, 'SS')
ibk_df = ibk_df.replace({'holder': '신한카드（주）'}, 'SH')
ibk_df = ibk_df.replace({'holder': '하나카드　주식회사'}, 'KEB')
ibk_df = ibk_df.replace({'holder': '비씨카드（주）'}, 'BC')             # 해외카드 승인된 BC가 입금될때

# details 컬럼에 'BC' 문자열 포함한 행들 추출해서 'holder' 컬럼 값 변경     # 국내 승인된 BC가 입금될떄
bc_expr = "details.str.endswith('BC')"          # BC로 끝나는 문자열
bc_lst = ibk_df.query(bc_expr).index.tolist()  # 조건 부합하는 열 값을 가진 행을 추출
ibk_df.loc[bc_lst, 'holder'] = 'BC'             # kb_lst 리스트에 포함된 모든 행의 'holder'컬럼 값 'KB'로 변경

# details 컬럼에 'KB' 문자열 포함한 행들 추출해서 'holder' 컬럼 값 변경
kb_expr = "details.str.startswith('KB')"       # KB로 시작하는 문자열
kb_lst = ibk_df.query(kb_expr).index.tolist()  # 조건 부합하는 열 값을 가진 행을 추출
ibk_df.loc[kb_lst, 'holder'] = 'KB'             # kb_lst 리스트에 포함된 모든 행의 'holder'컬럼 값 'KB'로 변경

# 카드사 입금이 아닌 라인 제외
ibk_df = ibk_df[ibk_df['holder'].isin(['HD', 'LT', 'SS', 'SH', 'KEB', 'BC', 'KB'])]
ibk_df

,date,receipts,details,holder
2,2022-06-30 10:49:27,2515449,777198403BC,BC
3,2022-06-30 10:35:36,277919,현611318397,HD
4,2022-06-30 08:47:40,864927,삼성카드838,SS
5,2022-06-30 07:25:07,158626,SHC0066754,SH
6,2022-06-30 06:28:29,958447,하나96043919,KEB
7,2022-06-30 06:03:31,487184,KB60757348,KB


In [3]:
# 'NH' 셋팅
# 농협의 xml 파일은 실제 내용은 html, 따라서 BeautifulSoup를 이용해서 읽어들임
from bs4 import BeautifulSoup
import re

# 농협 거래내역 xml 파일을 읽어서 Beautifuloup 객체를 만든다
# page = open('./nh20220629.xls', 'rt').read()
with open('./downdata/20220630/nh20220629.xls', 'rt') as page:
    soup = BeautifulSoup(page, 'html.parser')

# print(soup.prettify())

# # 컬럼 제목 추출
# colm_name_lst = soup.select('thead > tr > th')
# colm_name_lst = [name.get_text() for name in colm_name_lst])

# 내용 추출
content_soup = soup.select('td.se-td')   # 입출금내역 beautifulsoup 객체, 한 라인의각 컬럼을 리스트 모양으로 저장
row_lst = []                            # content list이며, DataFrame 생성에 사용
for content in content_soup:        # 각 컬럼을 looping
    # 정규 표현식에 적용되면 적용된 내용을 반환하고, 적용되지 않으면 원본 그대로 반환
    # text 주변의 공백을 제거한 내용 추출
    row_lst.append(re.sub('([0-9,]+) 원', '\\1', content.get_text().strip()))

# print(row_lst)

# content 리스트를 dataframe 변환
nh_df = pnds.DataFrame([row_lst], columns=['date', 'drawing', 'receipts', 'balance', 'holder'])

# 날짜 포맷 : '%Y-%m-%d %H:%M:%S' => 두 dataframe 통합 후에 정렬 및 날짜포맷 적용을 위함
nh_df['date'] = nh_df['date'].astype('datetime64[ns]', errors='ignore')    # date 포맷 변경을 위해서 datetimee 형으로 변환
nh_df['date'] = nh_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')     # 연산을 통해 포맷 변경 => 반환 타입은 일반 객체로 변경됨

# 천단위 ','를 없애고 금액 타입을 'int64'로 변경
nh_df['receipts'] = nh_df['receipts'].str.replace(',', '').astype('int64', errors='ignore')
# nh_df['balance'] = nh_df['balance'].str.replace(',', '').astype('int64', errors='ignore')

# 'details' 컬럼 추가하고, 불필요한 출금 컬럼 삭제
nh_df['details'] = 'NH11381135'
nh_df = nh_df[['date', 'receipts', 'details', 'holder']]

####
# 위 / 아래의 details 부분이 문제!!! details가 먼저 추가 된 상태에서는 모든 라인이 다 추출됨...column명 다시 확인할것!!!
####

# NH카드에서 입금된 라인의 holder 값을 'NH'로 바꿈
# details 컬럼에 'KB' 문자열 포함한 행들 추출해서 'holder' 컬럼 값 변경
str_expr = "holder.str.startswith('NH11381135')"       # NH로 시작하는 문자열
nh_lst = nh_df.query(str_expr).index.tolist()           # 조건 부합하는 열 값을 가진 행을 추출
nh_df.loc[nh_lst, 'holder'] = 'NH'                      # nh_lst 리스트에 포함된 모든 행의 'holder'컬럼 값을 'NH'로 변경

# NH카드 입금 라인만 추출
nh_df = nh_df[nh_df['holder'].isin(['NH'])]
nh_df

,date,receipts,details,holder
0,2022-06-29 07:23:05,1030204,NH11381135,NH


In [4]:
# 기업은행 데이타프레임(ibk_df)와 농협 데이타프레임(nh_df)를 합친다
bank_df = pnds.concat([ibk_df, nh_df], ignore_index=True)

# 2-4. 거래일시 기준 Sorting
bank_df = bank_df.sort_values(by=['date'])

# 2-5. 날짜 포맷 : '%Y-%m-%d'
bank_df['date'] = bank_df['date'].map(lambda str_data: str_data.split()[0], na_action='ignore') # 날짜 포맷 : '%Y-%m-%d'

# # DataFrame 저장
# target_date = (datetime.datetime.now() - datetime.timedelta(1)).strftime('%Y%m%d')  # 어제날짜의 폴더 이름
# base_dir = 'C:\\work\\pycard\\dtdata\\' + target_date
# now_date = bank_df.iloc[1]['date']      # 정렬된 상태에서의 첫번쨰 인덱스를 선택 (정렬 이전의 지료에서 1번 인덱스는 다음날의 자료일 가능성이 있음)
# file_name = '/dt_bank_' + now_date
# data_dir = base_dir + file_name
# with open(data_dir, "wb") as file:
#     pickle.dump(bank_df, file)

# 새로운 워크북/시트 생성
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl
from openpyxl.styles import Alignment

# bank_wb = openpyxl.Workbook()
# bank_sheet = bank_wb.active
# KICC 입금현황 엑셀 파일을 읽는다
bank_wb = openpyxl.load_workbook('./downdata/20220630/입금현황 · 일별 (2)_kicc.xlsx')
bank_sheet = bank_wb.active
row_count = bank_sheet.max_row      # 앞으로 추가할 Dataframe의 첫 라인을 강조('Pandas' style)하기 위한 참조

# 필요없는 셀 삭제
bank_sheet.delete_cols(2, 3)        # 3개 컬럼 삭제(사업자등록번호, 사업자상호, 가맹점번호)

#   Dataframe을 추가하기 전에 엑셀 파일에 맞게 컬럼을 조정한다
bank_df = bank_df[['holder', 'date', 'details', 'receipts']]        # 컬럼 순서 재조정
for idx in range(6):                                                # 컬럼 갯수 맞춤
    bank_df.insert(2, '--', '', allow_duplicates=True)

# openpyxl로 엑셀파일에 Datframe을 주입
for row in dataframe_to_rows(bank_df, index=False, header=True):    # index 제외
    bank_sheet.append(row)

# 각 컬럼의 width 설정
for idx in range(97, 97 + bank_sheet.max_column):           # 영문 'A'의 십진수 97...
    bank_sheet.column_dimensions[chr(idx)].width = 13.6

# 각 행의 height 설정
for row in range(bank_sheet.max_row):
    bank_sheet.row_dimensions[row].height = 18

# index와 column 이름을 강조하는 'Pandas' style
for cell in bank_sheet[1] + bank_sheet[row_count + 1]:      # KICC 입금현황 첫 줄과 추가한 Dataframe 첫 줄
    cell.style = 'Pandas'

# 추가한 Dataframe을 위하여 해당 컬럼의 셀 재정렬
for cell in bank_sheet['A'] + bank_sheet['B'] + bank_sheet['I']:
    cell.alignment = Alignment(horizontal='center', vertical='center')

# 'J'컬럼들의 숫자 포맷 셋팅
for rng in bank_sheet['J']:
    rng.number_format = '#,### '

# 엑셀 저장
try:
    dfdata_dir = './'
    xl_filename = 'my_bank' + '.xlsx'         # 저장파일 'df_bank_YYYYMMDD.xlsx'
    
    bank_wb.save(dfdata_dir + xl_filename)
except Exception as e:
    with open('./error.log', 'a') as file:
        file.write(
            f'[kicc_history.py - Making Excel] <{datetime.datetime.now()}> Pandas.ExcelWriter error ({xl_filename}) ===> {e}\n'
        )
    raise(e)

# 10-3. dataframe 그대로 excel로 저장 => 추후에 사용 가능할수 있음
# try:
#     xl_filename = df_filename + '.xlsx'         # 저장파일 'df_bank_YYYYMMDD.xlsx'
    
#     with pnds.ExcelWriter(dfdata_dir + xl_filename, mode='w', engine='openpyxl') as writer:
#         bank_df.to_excel(writer, sheet_name=f'{receipts_date}', index=False)
# except Exception as e:
#     with open('./error.log', 'a') as file:
#         file.write(
#             f'[kicc_history.py - Making Excel] <{datetime.datetime.now()}> Pandas.ExcelWriter error ({xl_filename}) ===> {e}\n'
#         )
#     raise(e)

c:\ProgramData\Anaconda3\envs\main\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
